In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import time
from configobj import ConfigObj
from glob import glob

In [2]:
# Create a posgresql database connection using settings from config file 

# Create a database connection using settings from config file
config='../db/config.ini'
# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = '243'
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '243':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                    user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                    host=conn_info["sqlhost"],
                                    port=conn_info["sqlport"],
                                    user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')
    
        con = psycopg2.connect(dbname=conn_info["dbname"],
                                host=conn_info["sqlhost"],
                                port=conn_info["sqlport"],
                                user=conn_info["sqluser"],
                                password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [19]:
def main4():
    ftName=['patientunitstayid']
    df7=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
    df5=pd.read_csv(os.getcwd()+'\\respiratorycharting_com.csv')
    df9=pd.read_csv(os.getcwd()+'\\dialysisday.csv')
    for i in range(0,30):
        ftName.append('respchart %02d'%(i))
    df0=pd.DataFrame(columns=ftName)
    count=0
    start=0
    end=0
    pid=[]
    for i in range(0,len(df7)):
        pid.append(df7['patientunitstayid'][i])
    for i in range(0,len(df9)):
        respchartday=getrespchartday(df7['patientunitstayid'][pid.index(df9['patientunitstayid'][i])],df7['Firstday'][pid.index(df9['patientunitstayid'][i])])
        daycount=0
        space_row = []
        print('%d/%d:%s %d'%(i,len(df9),df9['patientunitstayid'][i],respchartday),end='\r')
        for a in range(0,len(ftName)):
            space_row.append(np.nan)
        df0 = df0.append(pd.Series(space_row, index=df0.columns), ignore_index=True)
        df0['patientunitstayid'][count]=df9['patientunitstayid'][i]
        for j in range(0,30):
            if(respchartday==0):
                df0['respchart %02d'%(j)][count]=0
            elif(respchartday>0):
                df0['respchart %02d'%(j)][count]=df9['dialysisday %02d'%(j)][count]-respchartday+1
            if(pd.isna(df9['dialysisday %02d'%(j+1)][count])):
                break
        count+=1
    df0.to_csv(os.getcwd()+'\\respchartoffset.csv',index=False)
    

In [20]:
def getrespchartday(patientunitstayid,firstday):
    firstday=firstday-1440
    query = query_schema + """
    select * from respiratorycharting where patientunitstayid='{}' and respchartvaluelabel like '%PEEP%'
    and respchartoffset>{}
	order by respchartoffset
    """.format(int(patientunitstayid),firstday)
    df_respchartoffset=pd.read_sql_query(query, con)
    df_respchartoffset=df_respchartoffset[['respchartoffset','respchartvalue']]
    offset=[]
    if(len(df_respchartoffset)==0):
        return 0
    for i in range(0, len(df_respchartoffset)):
        df_respchartoffset['respchartvalue'][i]=float(df_respchartoffset['respchartvalue'][i])
        if(int(df_respchartoffset['respchartvalue'][i])<=0):
            continue
        else:
            for day in range(1,100):        
                if(firstday+1440*day>=df_respchartoffset['respchartoffset'][i]):
                    break;
            offset.append(day)
    
    try:
        return np.nanmin(offset)
    except TypeError:
        return 0
    except ValueError:
        return 0
    

In [21]:
def main2():
    ftName=['patientunitstayid']
    df7=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
    df5=pd.read_csv(os.getcwd()+'\\respiratorycharting_com.csv')
    df6=pd.read_csv(os.getcwd()+'\\vitalperiodic.csv')
    for i in range(0,30):
        ftName.append('dialysisday %02d'%(i))
    df0=pd.DataFrame(columns=ftName)
    count=0
    start=0
    end=0
    for i in range(0,len(df7)):
        #i=4
        dialysisday=[]
        dialysisoffset=[]
        dialysisday=getdialysisday(df7['patientunitstayid'][i],df7['Firstday'][i])
        dialysisday=np.unique(dialysisday)
        dialysisoffset=getdialysisoffset(df7['patientunitstayid'][i],df7['Firstday'][i])
        dialysisoffset=np.unique(dialysisoffset)
        if(len(dialysisday)>0 and len(dialysisoffset)>0):
            daycount=0
            space_row = []
            print('%d/%d:%s'%(i,len(df7),df7['patientunitstayid'][i]),end='\r')
            for a in range(0,len(ftName)):
                space_row.append(np.nan)
            df0 = df0.append(pd.Series(space_row, index=df0.columns), ignore_index=True)
            df0['patientunitstayid'][count]=df7['patientunitstayid'][i]
            for j in range(0,len(dialysisday)):
                df0['dialysisday %02d'%(daycount)][count]=dialysisday[j]
                daycount+=1
            count+=1
    df0.to_csv(os.getcwd()+'\\dialysisday.csv',index=False)

In [22]:
def celllabel(patientunitstayid,firstday):
    query = query_schema + """
    select distinct intakeoutputoffset
    from intakeoutput
    where patientunitstayid = '{}' and celllabel like '%dialysis%'
    order by intakeoutputoffset
    """.format(int(patientunitstayid))
    df_IO=pd.read_sql_query(query, con)
    offset=[]
    offset2=[]
    if(len(df_IO)==0):
        return None
    else:
        for i in range (0,len(df_IO)):
            offset.append(df_IO['intakeoutputoffset'][i])
    hemodialysisdayend=-1000000
    flag=0
    for i in range(0,len(offset)):   
        if (offset[i] >hemodialysisdayend):
            if (offset[i]>firstday):
                if(offset[i]-firstday>1440):
                    for j in range(1,100):
                        temp=firstday+1440*j
                        if(temp>offset[i]):
                            hemodialysisdayend=temp
                            break
                else:
                    hemodialysisdayend=firstday+1440
            elif (offset[i]<firstday):
                hemodialysisdayend=firstday
            offset2.append(hemodialysisdayend-1440)
    return offset2

In [23]:
def getdialysisoffset(patientunitstayid,firstday):
    query = query_schema + """
    select distinct treatmentoffset
    from treatment
    where patientunitstayid = '{}' and treatmentstring like '%dialysis%' and activeupondischarge = 'False'
    order by treatmentoffset
    """.format(int(patientunitstayid))
    df_treatment=pd.read_sql_query(query, con)
    offset=[]
    offset2=[]
    if(len(df_treatment)==0):
        df_celllabel=celllabel(patientunitstayid,firstday)
        return df_celllabel
    for i in range (0,len(df_treatment)):
        offset.append(df_treatment['treatmentoffset'][i])
    hemodialysisdayend=-1000000
    flag=0
    for i in range(0,len(offset)):   
        if (offset[i] >hemodialysisdayend):
            if (offset[i]>firstday):
                if(offset[i]-firstday>1440):
                    for j in range(1,100):
                        temp=firstday+1440*j
                        if(temp>offset[i]):
                            hemodialysisdayend=temp
                            break
                else:
                    hemodialysisdayend=firstday+1440
            elif (offset[i]<firstday):
                hemodialysisdayend=firstday
            else:
                hemodialysisdayend=firstday+1440
            offset2.append(hemodialysisdayend-1440)
    return offset2

In [24]:
def getdialysisday(patientunitstayid,firstday):
    query = query_schema + """
    select distinct treatmentoffset
    from treatment
    where patientunitstayid = '{}' and treatmentstring like '%dialysis%' and activeupondischarge = 'False'
    order by treatmentoffset
    """.format(int(patientunitstayid))
    df_treatment=pd.read_sql_query(query, con)
    offset=[]
    if(len(df_treatment)==0):
        df_celllabel=celllabel(patientunitstayid,firstday)
        for i in range(0, len(df_celllabel)):
            for day in range(0,100):        
                if(firstday+1440*day>df_celllabel[i]):
                    break;
            if(day>27):
                continue
            offset.append(day+1)
        return offset
    for i in range(0, len(df_treatment)):
        for day in range(0,100):        
            if(firstday+1440*day>df_treatment['treatmentoffset'][i]):
                break;
        if(day>27):
            continue
        offset.append(day+1)
    return offset
    

In [25]:
def main3():
    df7=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
    df5=pd.read_csv(os.getcwd()+'\\respiratorycharting_com.csv')
    df6=pd.read_csv(os.getcwd()+'\\vitalperiodic.csv')
    ftName0=['patientunitstayid','dialysisstart','dialysisend']
    count=0
    df0=pd.DataFrame(columns=ftName0)
    for i in range(0,len(df7)):
        dialysisday=[]
        dialysisoffset=[]
        dialysisday=getdialysisday(df7['patientunitstayid'][i],df7['Firstday'][i])
        dialysisday=np.unique(dialysisday)
        dialysisoffset=getdialysisoffset(df7['patientunitstayid'][i],df7['Firstday'][i])
        dialysisoffset=np.unique(dialysisoffset)
        if(len(dialysisday)>0 and len(dialysisoffset)>0): 
            for j in range(0,len(dialysisday)):
                print('%d/%d:%s'%(i,len(df7),df7['patientunitstayid'][i]),end='\r')
                if(df5['target D%02d'%(dialysisday[j])][i]==1):
                    space_row = []
                    for a in range(0,len(ftName0)):
                        space_row.append(0)
                    df0 = df0.append(pd.Series(space_row, index=df0.columns), ignore_index=True)
                    df0['patientunitstayid'][count]=df7['patientunitstayid'][i]
                    vitotal=0
                    start=0
                    end=0
                    for a in range(start,len(df6)):
                        if(df6['patientunitstayid'][a]==df0['patientunitstayid'][count]
                           and dialysisoffset[j]<df6['observationoffset'][a]<=dialysisoffset[j]+4320):
                            vitotal+=1
                            if(vitotal==1):
                                start=a
                        if((df6['patientunitstayid'][a]==df0['patientunitstayid'][count] 
                           and df6['observationoffset'][a]>dialysisoffset[j]+4320) or df6['patientunitstayid'][a]!=df0['patientunitstayid'][count] and vitotal!=0):
                            end=a
                            break
                    df0['dialysisstart'][count]=start
                    df0['dialysisend'][count]=end
                    count+=1
    df0.to_csv(os.getcwd()+'\\dialysisoffset.csv',index=False)
                    

In [27]:
if __name__ is '__main__':
    #main2()#create dialysis day csv file 
    #main3()#create dialysis offset csv file 
    main4()#create ventilation offset csv file 

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
